In [ ]:
#Senya Stein
#CSCI 101 
#Section B

#!/usr/bin/env python
# coding: utf-8

#license = Apache 2.0 open source code

#Extracts date, lat, lon from Saildrone data 
#Code adapted from Chelle Gentemann, Johannes Karstensen 

#import relevant libraries and functions
import numpy as np
from ipyleaflet import GeoJSON
import requests

def get_key(file_name):
    myvars = {}
    with open(file_name) as myfile:
        for line in myfile:
            name, var = line.partition("=")[::2]
            myvars[name.strip()] = str(var).rstrip()
    return myvars

file_key = "C:/Users/gentemann/Google Drive/f_drive/secret_keys/saildrone_eurec4a_v2.txt"
saildrone_key = get_key(file_key)

In [ ]:
#web requests using requests
wfs_base_url = "https://maps.geomar.de/geoserver/OceanEddies/wfs" #server location
requested_layer = "OceanEddies:current_positions"
#dictionary of data
requests_parameters = {
                        'service': 'wfs',
                        'version': '1.0.0',
                        'request': 'getFeature',
                        'typename': requested_layer,
                        #filter the results on the server
                       # 'CQL_FILTER': "type like 'Saildrone'",  # OCG CQL filter
                        'viewparams': "maxage:48",  # parameters for custom view on geoserver: set max age of position to 48h 
                        "outputformat": "application/json"
                      }
response = requests.get(wfs_base_url, params=requests_parameters)
print(f'use the following link for your query: {response.url}')

wg_positions = GeoJSON(data=response.json(), hover_style={'fillColor': 'red'}, name='WaveGlider Positions') 
#print(response.json())


#assign the web response to dict_assets (a dictionary containing all the assets)

dict_assets = response.json()
dict_assets

#assign dict values to variables 
dict_assets['features'][0]['properties']['obs_timestamp']
dict_assets['features'][0]['properties']['type']
dict_assets['features'][0]['geometry']['coordinates']
date = dict_assets['features'][0]['properties']['obs_timestamp']                          

#converts time to Unix timestamp

def convertunix(date):
    
    #we don't need the Z at the end
    date = np.datetime64(date.replace('Z', ''))

    timestamp = (date - np.datetime64('1970-01-01T00:00:00')) / np.timedelta64(1, 's')
    return timestamp


#extracts latitude
def coordlat(coords):
    coords = dict_assets['features'][0]['geometry']['coordinates']
    lat = coords[0]
    return lat
    
#extracts longitude
def coordlon(coords):
    coords = dict_assets['features'][0]['geometry']['coordinates']
    lon = coords[1]
    return lon   

#finds number of assets to extract
number_of_assets = len(dict_assets['features'])
print('Asset Number = ' + str(number_of_assets))

asset_dictionary = {} #this will store the final lat lon and time


i,isave = 0,0

#adds each asset to dictionary to output
while i < number_of_assets:

    timestamp = convertunix(dict_assets['features'][i]['properties']['obs_timestamp']) #timestamp
    aplatform_id = dict_assets['features'][i]['properties']['platform_id'] #platform id
    atype = dict_assets['features'][i]['properties']['type'] #platform type
    aname = dict_assets['features'][i]['properties']['name'] #platform type
    rlat = coordlon(dict_assets['features'][i]['geometry']['coordinates']) #latitude
    rlon = coordlon(dict_assets['features'][i]['geometry']['coordinates']) #longitude
    rsog = dict_assets['features'][i]['properties']['speed_over_ground'] 
    rheading = dict_assets['features'][i]['properties']['heading'] 
    i+=1
    
    if atype=='Saildrone':
        continue
    
    if rsog=='None':
        rsog=0.
    if rheading=='None':
        rheading=0.
    
    #final dictionary of all relevant information (frome above)
    asset_dictionary[isave] = { 
      "type" : atype,
      "name" : aname,
      "mmsi" : aplatform_id,
      "latitude" : rlat,
      "longitude" : rlon,
      "timestamp" : np.int(timestamp) ,
      "hdg": rheading,
      "sog": rsog #,
     # "token": saildrone_key['token']
      }
    #print(asset_dictionary[i])
    isave+=1


In [ ]:
print('number of items in dict (type, id, geometry, geometry_name, properties....):', len(dict_assets))
print('number of assest:',len(dict_assets['features']))

In [ ]:
#'Content-Type: application/json' --header 'Accept: application/json' --header 'authorization: 1030'
#get token
#payload={'key': saildrone_key['key'], 'secret':saildrone_key['secret']}
for iasset in asset_dictionary:
    payload=asset_dictionary[iasset] #{'key': saildrone_key['key'], 'secret':saildrone_key['secret']}
    headers={'Content-Type':'application/json', 'Accept':'application/json','authorization':saildrone_key['token']}
    url = 'https://developer-mission.saildrone.com/v1/ais'
    res = requests.post(url, json=payload, headers=headers)


In [ ]:
#do NOT commit any changes with payload printed out because it has token
#payload